This is a note for [3D Coordinates and Representations of Rotations (Cyrill Stachniss, 2020)](https://youtu.be/YXGUGSAv09A)

In [1]:
using LinearAlgebra
using Random
using Distributions

In [2]:
using NBInclude
@nbinclude("../Basics/homogeneous_coordinates.ipynb");
#Import the EuclideanCoord"

### Two properties of rotation matrix:
1) det(r) == 1;
2) inv(r) == transpose(r)

In [3]:
function assert_rotation_matrix(m::AbstractMatrix)
    @assert det(m) ≈ 1 "det is $(det(m))"
    @assert transpose(m) * m ≈ I "inv is $(inv(m)) and transpose is $(transpose(m))"
    #@assert all(norm(m, 1) .≈ 1) # this can be proved from the 2nd property
end

struct Rotation{T<:Real}
    matrix::AbstractMatrix{T}
    function Rotation(m::AbstractMatrix{T}) where {T}
        assert_rotation_matrix(m)
        new{T}(m)
    end
end

# 2d toy example (2d euler to rotation matrix)
function Rotation(θ::Real)::Rotation
    return Rotation([cos(θ) -sin(θ); sin(θ) cos(θ)])
end;

### Euler Angle
1) The order of yaw, pith, and roll **matters**\
2) **Sigularity**: cosβ=0 i.e. β=+-90 ("Glimbal Lock")\
3) Euler angles are unique given a rotation matrix, if the angle is in range -π to π.
4) The rotation matrix is unique given euler angles

In [4]:
struct EulerAngle{T<:Real}
    θ::AbstractVector{T}
end

#Rotation Matrix to Euler angles
function EulerAngle(r::Rotation)::EulerAngle
    if r.matrix[3,2] == 0 && r.matrix[3,3] == 0
        β = asin(-r.matrix[3,1])
        α = 0
        if r.matrix[3,1] < 0 #singularity #2
            γ = atan(-r.matrix[1,2], r.matrix[1,3])
        else
            γ = atan(-r.matrix[1,2], -r.matrix[1,3])
        end
    else
        α=atan(r.matrix[3,2], r.matrix[3,3])
        β=atan(-r.matrix[3,1], √(r.matrix[3,2]^2+r.matrix[3,3]^2))
        γ=atan(r.matrix[2,1], r.matrix[1,1])
    end
    return EulerAngle([α, β, γ])
end
#Euler Angles to Rotation Matrix
function Rotation(angle::EulerAngle)::Rotation
    rx = [1 0 0;
        0 cos(angle.θ[1]) -sin(angle.θ[1]);
        0 sin(angle.θ[1]) cos(angle.θ[1])]
    ry = [cos(angle.θ[2]) 0 sin(angle.θ[2]);
        0 1 0;
        -sin(angle.θ[2]) 0 cos(angle.θ[2])]
    rz = [cos(angle.θ[3]) -sin(angle.θ[3]) 0;
        sin(angle.θ[3]) cos(angle.θ[3]) 0;
        0 0 1]
    return Rotation(rz*ry*rx) #Yaw, Pitch, Roll
end;

### Axis Angle
1) **Singularity**: θ=0 (between axis angle and rotation matrix)

In [5]:
struct AxisAngle{T<:Real}
    r::AbstractVector{T}
end

#Rotation Matrix to Axis Angle
function AxisAngle(r::Rotation)::AxisAngle
    a = - [r.matrix[2,3]-r.matrix[3,2], r.matrix[3,1]-r.matrix[1,3], r.matrix[1,2]-r.matrix[2,1]]
    a_norm = norm(a)
    r_trace = tr(r.matrix)
    θ = atan(a_norm, r_trace-1)
    if a_norm == 0 #singularity #1
        return AxisAngle(a)
    end
    AxisAngle(θ*a/a_norm)
end

#Axis Angle to Rotation Matrix
function Rotation(a::AxisAngle)::Rotation
    θ = norm(a.r)
    if θ == 0
        return Rotation(Matrix(I,3,3)) #singularity #1
    end
    Sθ = [0 -a.r[3] a.r[2]; a.r[3] 0 -a.r[1]; -a.r[2] a.r[1] 0]
    Sr = Sθ / θ
    S_r = I + sin(θ)*Sr + (1-cos(θ))*(Sr)^2
    Rotation(S_r)
end;

### Quaternion
1) Both Euler Angles and Axis angle has singularities and discountinuities (2π and 0 are the same, this may screw up the gradients).
2) Similar to Rotation Matrix, Quaternion has no singularities or discountinuities
3) Quaternion only has 4 parameters compared to 9 parameters in Rotation Matrix. Quaternions are almost minimal

In [6]:
struct Quaternion{T<:Real}
    real::T
    imag::AbstractVector{T}
end

import Base: +,*
import LinearAlgebra.inv
function +(left::Quaternion, right::Quaternion)::Quaternion
    Quaternion(left.real+right.real, left.imag+right.imag)
end

function *(left::Quaternion, right::Quaternion)::Quaternion #compose two rotation
    imag = right.real.*left.imag + left.real*right.imag + cross(left.imag, right.imag)
    Quaternion(left.real*right.real-dot(left.imag,right.imag), imag)
end

function inv(q::Quaternion)
    Quaternion(q.real, -q.imag)
end

function Quaternion(a::AxisAngle)::Quaternion
    θ = norm(a.r)
    r = a.r/θ
    Quaternion(cos(θ/2), sin(θ/2).*r)
end;

In [7]:
function rotate(p::EuclidianCoord, r::Rotation)::EuclidianCoord
    return EuclidianCoord(r.matrix * p.coord)
end
function rotate(p::EuclidianCoord, q::Quaternion)::EuclidianCoord
    pq = Quaternion(0.0, p.coord)
    p_result = q * pq * inv(q)
    #@assert p_result.real ≈ 0 "$p_result" #i is very small, but may not pass the approx
    return EuclidianCoord(p_result.imag)
end
function rotate(p::EuclidianCoord, r::AxisAngle)::EuclidianCoord
    return rotate(p, Rotation(r))
end;

In [8]:
function scale(p::EuclidianCoord, λ::Real)
    return EuclidianCoord(p.coord.+λ)
end;

Test cases for euler, axis angle, and quaternion

In [9]:
function assert_euler(θ::AbstractVector)
    θs = EulerAngle(θ)
    rot = Rotation(θs)
    θs_back = EulerAngle(rot)
    rot_again = Rotation(θs_back)
    @assert rot.matrix ≈ rot_again.matrix
    #@assert θs.θ ≈ θs_back.θ  "$θs, $(θs_back.θ)" #Euler angles are unique if the input is in range -π/2 to π/2 and β != +-90
end
assert_euler(rand(Uniform(-π/2,π/2), (3)))
assert_euler([2, π/2, 1])

In [10]:
function assert_axis_angle(v::AbstractVector)
    ro = Rotation(AxisAngle(v))
    axisangle = AxisAngle(ro)
    ro_again = Rotation(axisangle)
    @assert ro.matrix ≈ ro_again.matrix
    @assert v ≈ axisangle.r "$v, $axisangle"
end
θ = rand(Uniform(-π,π), (1))
v = rand(Uniform(0,1), (3))
assert_axis_angle(θ .* v / norm(v))
assert_axis_angle([π,0,0])

In [11]:
θ1 = rand(Uniform(-π,π), (1))
v1 = rand(Uniform(0,1), (3))
a = AxisAngle(θ1 .* v1 / norm(v1))
p = EuclidianCoord(rand(Uniform(0,1), (3)))
p1 = rotate(p, a)
p2 = rotate(p, Quaternion(a))
p3 = rotate(p2, inv(Quaternion(a)))
@assert p1.coord ≈ p2.coord "$p1, $p2"
@assert p.coord ≈ p3.coord "$p, $p3"